In [75]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import json
import random
from PIL import Image
from sklearn.neighbors import NearestNeighbors

In [76]:
video_root = './v'
data_root = '../../data'
fit_data_root = '../../target'

if not os.path.exists(data_root):
    os.makedirs(data_root)

In [77]:
# shape of library img (SAO se3 frames)
W = 1280
H = 720

# load library
with open('./rgb_avr_hist_edge.json', 'r', encoding='utf-8') as f:
    avr_RGB_data = json.load(f)
    
lib_RGB = list(np.array(v) for v in avr_RGB_data.values()) # unpack to ndarrays
lib_serials = list(avr_RGB_data.keys())

In [78]:
nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(lib_RGB)

In [79]:
def get_fitted_target(serial: int):
	target = cv2.imread(f'{fit_data_root}/{serial}.jpg')
	
	h, w, _ = target.shape
	h_prime = round(H / W * w)
	return cv2.resize(target, (w, h_prime))

In [ ]:
W_SIZE = 200
H_SIZE = 200

def subdivide(t):
	subs = []

	height, width, channels = t.shape

	w_sub = width / W_SIZE
	h_sub = height / H_SIZE

	for ih in range(H_SIZE):
		for iw in range(W_SIZE):
			x = w_sub * iw 
			y = h_sub * ih

			sub = t[int(y):int(y+h_sub), int(x):int(x+w_sub)]
			subs.append(sub)

	return subs

In [92]:
# 色彩特徵維度較低
color_bins = [4, 4, 4]   # RGB 各通道 4 分區，共 64 維
edge_bins = 16           # 輪廓方向分成 16 區
edge_weight = 2.0        # 邊緣特徵的權重


def get_features(image_rgb):
	gray = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)

	# ===== 🎨 顏色特徵 (RGB 3D Histogram) =====
	hist = cv2.calcHist([image_rgb], [0, 1, 2], None, color_bins, [0, 256]*3)
	hist = cv2.normalize(hist, hist).flatten()  # shape: (64,)

	avr_rgb = [round(np.mean(c)) for c in cv2.split(image_rgb)]

	# ===== 📐 邊緣方向特徵 (Gradient Orientation Histogram) =====
	sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
	sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
	magnitude = np.sqrt(sobelx**2 + sobely**2)
	orientation = np.arctan2(sobely, sobelx)  # [-π, π]

	# 計算方向分布直方圖（以 magnitude 加權）
	orientation_hist, _ = np.histogram(
		orientation,
		bins=edge_bins,
		range=(-np.pi, np.pi),
		weights=magnitude
	)
	orientation_hist = orientation_hist / (np.sum(orientation_hist) + 1e-5)  # normalize

	# combine features into ndarray
	return np.concatenate([
		avr_rgb,
		hist,
		orientation_hist * edge_weight
	]) # shape: 3 + 4^3 + 16 = 83

def get_subdivide_RGB(subs):
	data = {}
	for i, img in enumerate(subs):
		# img comes in RGB
		# data[i] = [round(np.mean(c)) for c in cv2.split(img)]

		data[i] = get_features(img)

	return data

In [93]:
def select_candidates(t_RGB):
	_, indices = nbrs.kneighbors(t_RGB)

	selected_serial = []
	for ind in indices:
		ind = ind.tolist()
		fit_num = random.sample(ind, 1)[0]
		fit_serial = lib_serials[fit_num]
		selected_serial.append(fit_serial)

	return selected_serial

In [94]:
# thumbnail & output shape settings

thumb_width, thumb_height = W / W_SIZE * 5, H / H_SIZE * 5
grid_width = round(W_SIZE * thumb_width)
grid_height = round(H_SIZE * thumb_height)

print(thumb_width, thumb_height)
print(grid_width, grid_height)

thumb_width, thumb_height = round(thumb_width), round(thumb_height)

16.0 9.0
6400 3600


In [95]:
def load_image(serial):
	img_path = f"{data_root}/{serial}.jpg"
	
	try:
		img = Image.open(img_path).convert("RGB")
		return img.resize((thumb_width, thumb_height), Image.Resampling.LANCZOS)
	except Exception as e:
		print(img_path)
		print(e)

In [96]:
img_buffer = {}

def get_buffer(serial):
	img = img_buffer.get(serial, None)
	if img is None:
		img_buffer[serial] = load_image(serial)
		return img_buffer[serial]
	else:
		return img
	
def clear_buffer():
	for k, v in img_buffer.items():
		v.close()

In [97]:
def gen_result(candidates):
	composite_image = Image.new("RGB", (grid_width, grid_height))

	for i, serial in enumerate(candidates):
		x = (i % W_SIZE) * thumb_width
		y = (i // W_SIZE) * thumb_height

		composite_image.paste(get_buffer(serial), (round(x), round(y)))

	return composite_image

In [98]:
output_root = '../../result_lowres'

In [99]:
def target_workflow(target):
	t = get_fitted_target(target)
	t = cv2.cvtColor(t, cv2.COLOR_BGR2RGB)

	subs = subdivide(t)

	t_RGB_data = get_subdivide_RGB(subs)
	t_RGB = list(t_RGB_data.values())
	# t_serials = list(t_RGB_data.keys())

	candidates = select_candidates(t_RGB)
	result = gen_result(candidates)

	with open(f'{output_root}/{target}.jpg', 'w+') as f:
		result.save(f, "JPEG")

	result.close()

In [100]:
for i in range(1, 5 + 1):
    print(f"processing target {i}")
    target_workflow(i)

processing target 1
processing target 2
processing target 3
processing target 4
processing target 5


In [101]:
# img_buffer = {}